## 나이브 베이즈 모델로 분류하기
- 기존까지는 휴리스틱한 방식으로 weight를 더 주고 벡터 공간에 임베딩한 후 cosine 유사도를 통해 분류를 수행했다.
- 본 노트북에서는 빈도를 나이브 베이즈 모델로 분류를 수행해 본다

In [50]:
trainingSet = [
    (1, "Chinese Beijing Chinese", "yes"),
    (2, "Chinese Chinese Shanghi", "yes"),
    (3, "Chinese Macao", "yes"),
    (4, "Tokyo Japan Chinese", "no")
]

testSet = [
    (5, "Chinese Tokyo Japan") # 'Chinese'의 빈도에 따라 결과가 달라지게 된다. 
]

In [51]:
trainingSetYes = [d for d in trainingSet if d[2] == "yes"]
trainingSetNo = [d for d in trainingSet if d[2] == "no"]

In [52]:
trainingSetYes, trainingSetNo

([(1, 'Chinese Beijing Chinese', 'yes'),
  (2, 'Chinese Chinese Shanghi', 'yes'),
  (3, 'Chinese Macao', 'yes')],
 [(4, 'Tokyo Japan Chinese', 'no')])

In [53]:
nYes = len(trainingSetYes)
nNo = len(trainingSetNo)
N = nYes + nNo

priorYes = nYes / N # yes에 대한 prior
priorNo = 1- priorYes
print(priorYes), print(priorNo)

0.75
0.25


(None, None)

## Trainset에 대한 확률 구하기

In [54]:
from collections import defaultdict

tokensYes = defaultdict(int)
tokensNo = defaultdict(int)

for d in trainingSetYes:
    for t in d[1].lower().split(): # 단어가 하나씩 생김.
        tokensYes[t] += 1
        
for d in trainingSetNo:
    for t in d[1].lower().split(): # 단어가 하나씩 생김.
        tokensNo[t] += 1
        
# lexicon 생성(set으로 중복 제거 후 list로 묶기)
vocabulary = list(set(list(tokensYes.keys()) + list(tokensNo.keys())))

In [55]:
# conditional probability 계산
from math import log
cpYes = defaultdict(float) # conditional probability이면서 Yes인 것들
cpNo = defaultdict(float) # conditional probability이면서 Yes인 것들
K = 0.5 # K를 통해 스무딩하는 방법
B = len(vocabulary) # 스무딩
sumYes = sum(tokensYes.values())
sumNo = sum(tokensNo.values())

for t in vocabulary:
   # 특정 클래스 내 document에 속하는 단어들 중 몇번 나왔는가를 구한 후 MLE 추정
   # 로그를 취함으로서 덧셈 형태로 바뀌므로 +=를 함. (왜 log를 씌우는지는 원노트 참고)
#     스무딩 방법 1)
#     cpYes[t] += log((tokensYes[t] + 1) / (sumYes + B)) # 로그를 씌운 확률값
#     cpNo[t] += log((tokensNo[t] + 1) / (sumNo + B))
#     스무딩 방법 2)
    cpYes[t] += log((tokensYes[t] + K) / (sumYes + K*2)) # 일반적으로 K를 통해 절반의 확률을 유도한다. 
    cpNo[t] += log((tokensNo[t] + K) / (sumNo + K*2))

    print(t)
    print("Yes | log({0}+1/{1}+{2}) = {3:.4f}".format(tokensYes[t], sumYes, B, cpYes[t]))
    print("No | log({0}+1/{1}+{2}) = {3:.4f}".format(tokensNo[t], sumNo, B, cpNo[t]))
    

chinese
Yes | log(5+1/8+6) = -0.4925
No | log(1+1/3+6) = -0.9808
beijing
Yes | log(1+1/8+6) = -1.7918
No | log(0+1/3+6) = -2.0794
tokyo
Yes | log(0+1/8+6) = -2.8904
No | log(1+1/3+6) = -0.9808
macao
Yes | log(1+1/8+6) = -1.7918
No | log(0+1/3+6) = -2.0794
shanghi
Yes | log(1+1/8+6) = -1.7918
No | log(0+1/3+6) = -2.0794
japan
Yes | log(0+1/8+6) = -2.8904
No | log(1+1/3+6) = -0.9808


## Testset에 대한 확률 구하기

In [57]:
from math import exp

for d in testSet:
    scoreYes = log(priorYes)
    scoreNo = log(priorNo)
    
    for t in d[1].lower().split(): # 단어가 하나씩 생김.
        scoreYes += cpYes[t] 
        scoreNo += cpNo[t]
    
    # 분류
    if scoreYes > scoreNo:
        print("Yes")
    else:
        print("No")
        
    print(exp(scoreYes), exp(scoreNo))

No
0.001414609053497941 0.01318359375
